In [1]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

0. Batch process

In [2]:
# from dl.dl_prediction import predict_yield_from_img

In [3]:
# yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
# out_path = '../output/'
# img_root_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/'
# # path_list = ['LIRF20220628_DOY179_extracted_filled', 'LIRF20220708_DOY189_extracted_filled', 'LIRF20220716_DOY197_extracted_filled', \
# path_list = ['LIRF20220628_DOY179_extracted_filled', 'LIRF20220708_DOY189_extracted_filled', 'LIRF20220716_DOY197_extracted_filled','LIRF20220720_DOY201_extracted_filled', 'LIRF20220916_DOY259_extracted_filled', 'LIRF20220926_DOY269_extracted_filled']

# for path_i in path_list:
#     img_path = img_root_path+path_i
#     print(img_path)

#     predict_yield_from_img(yield_file, img_path, out_path, True, True)

In [4]:
import numpy as np

from sklearn.model_selection import train_test_split

In [5]:
import torch
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from dataset import select_data_and_yield_list
from dl.dl_dataset import CornDataset
from dl.model import ResNetRegression
from dl.train import train_with_cross_validation, train, validate, data_transform

In [7]:
yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220610_DOY161_extracted_filled'
# out_path = '../output/DOY161/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled'
# out_path = '../output/DOY167/'
img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
out_path = '../output/DOY186/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
# out_path = '../output/DOY201/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220916_DOY259_extracted_filled'
# out_path = '../output/DOY259/'


In [8]:
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled'
# out_path = '../output/DOY179/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220716_DOY197_extracted_filled'
# out_path = '../output/DOY197/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = '../output/DOY269/'


In [9]:
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled'
# out_path = '../output/DOY189/'

In [10]:

if not os.path.exists(out_path):
    # If it doesn't exist, create the directory and any missing parent directories
    os.makedirs(out_path)
    print(f"Directory '{out_path}' created successfully.")
else:
    print(f"Directory '{out_path}' already exists.")


name_tag = img_path[-23:-17]
# print(name_tag)

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['evi']
for keyword in key_word_list:

    # selection = ['Pioneer'] # 
    selection = None
    # selection = 'Pioneer Deficit' 
    # selection = 'Pioneer Full'
    pioneer_img_list, pioneer_yield_list, irrigate_type_list = select_data_and_yield_list(
        img_path, yield_file, key_word = keyword, crop_type_select=selection)
    
    # for suffix_list in suffix_list_list:
    total_size = len(pioneer_yield_list)
    train_size = int(0.8 * total_size)  # 80% for training
    test_size = int(0.2 * total_size)   # 20% for validation
    
    # train_val_dataset = CornDataset(pioneer_img_list, pioneer_yield_list)
    
    # Use train_test_split to split the indices into training and testing sets
    train_indices, test_indices = train_test_split(range(len(pioneer_img_list)), test_size=test_size, random_state=39)

    train_val_dataset = CornDataset([pioneer_img_list[i] for i in train_indices], [pioneer_yield_list[i] for i in train_indices], transform=data_transform())
    test_dataset = CornDataset([pioneer_img_list[i] for i in test_indices], [pioneer_yield_list[i] for i in test_indices])

    

Directory '../output/DOY197/' already exists.


2. 

In [ ]:


in_channel = 5
num_epochs = 200
batch_size = 32

# Initialize an empty list to store fold-wise performance
fold_accuracies = []

# Initialize a new model for each fold
# model = CNNRegression(in_channel)
model = ResNetRegression(in_channel, 1)
# model = EncoderCNN(in_channel, 1)
print(device)
model.to(device)

criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
# optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# optimizer.zero_grad()

model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion)

model_name = "path/model_" + model.__class__.__name__+ img_path[-24:-17] +"_Batch=" +str(batch_size) + "_state.pth"
torch.save(model.state_dict(), model_name)

3. 

In [11]:
# model_name = "path/model_ResNetRegression_DOY161_32_state.pth"
# model_name = "path/model_ResNetRegression_DOY161_batch = 64_state.pth"
# Load the model state dictionary
# loaded_model = EncoderCNN(in_channel,1)
model_name = "path/pioneer/model_ResNetRegression_DOY186_Batch=32_state.pth"
in_channel = 5
num_epochs = 200
batch_size = 32

loaded_model = ResNetRegression(in_channel, 1)
loaded_model.load_state_dict(torch.load(model_name))
loaded_model.to(device)

ResNetRegression(
  (resnet): ResNet(
    (conv1): Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [12]:

criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
test_accuracy, test_prediction = validate(loaded_model, test_dataset,criterion, batch_size = batch_size, is_return_output = True)

In [13]:
test_accuracy = np.sqrt(np.mean(test_accuracy))
print(f'validation mse is {test_accuracy}')

validation mse is 96.81622239051032


In [ ]:
print(test_prediction)

In [ ]:
print(len(test_prediction))

In [14]:
from ml.ml_predict import plot_result_separate
yield_data = np.array(pioneer_yield_list)
irrigate_data = np.array(irrigate_type_list)

test_truth = yield_data[test_indices]


name_tag = img_path[-23:-17]
out_name = name_tag + ' ' + 'Pioneer' + ' ' 
out_name = out_name + keyword[:-11] 
out_name = out_name + ' CNN_test'

plot_result_separate(np.array(test_truth), np.array(test_prediction), test_indices, irrigate_data, out_name)

DOY197 Pioneer Ref CNN_test  r-squared = -3.0420502949477335, rmse = 96.8259434403301
